In [1]:
import random
import time
import numpy as np
import math 
import cv2
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO 
from typing import Callable
import os
import carla

2024-05-05 13:03:30.049218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 13:03:30.119684: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-05 13:03:30.441484: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cud

In [2]:
SECONDS_PER_EPISODE = 25

N_CHANNELS = 3
HEIGHT = 250
WIDTH = 250

FIXED_DELTA_SECONDS = 0.2

SHOW_PREVIEW = True

In [3]:
class CarEnv(gym.Env):
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = WIDTH
    im_height = HEIGHT
    front_camera = None
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4
    def __init__(self):
        super(CarEnv, self).__init__()
        
        self.action_space = spaces.MultiDiscrete([ 9, 4 ])

        self.observation_space = spaces.Box(low=0.0, high=255.0,
                                            shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)
        
        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(4.0)
        self.world = self.client.get_world()

        self.settings = self.world.get_settings()
        self.settings.no_rendering_mode = True
        #self.settings.synchronous_mode = False
        self.settings.fixed_delta_seconds = FIXED_DELTA_SECONDS
        self.world.apply_settings(self.settings)
        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter("model3")[0]
        
    def reset(self, seed=None, options=None):
        self.collision_hist = []
        self.actor_list = []
        self.transform = random.choice(self.world.get_map().get_spawn_points())

        self.vehicle = None
        while self.vehicle is None:
            try:
        # connect
                self.vehicle = self.world.spawn_actor(self.model_3, self.transform)
            except:
                pass
        self.actor_list.append(self.vehicle)
        self.initial_location = self.vehicle.get_location()
        self.sem_cam = self.blueprint_library.find('sensor.camera.semantic_segmentation')
        self.sem_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.sem_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.sem_cam.set_attribute("fov", f"90")


        camera_init_trans = carla.Transform(carla.Location(z=self.CAMERA_POS_Z,x=self.CAMERA_POS_X))
        self.sensor = self.world.spawn_actor(self.sem_cam, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(2) #not detect a collision when the car spawns/falls from sky.
        # showing camera at the spawn point
        if self.SHOW_CAM:
            cv2.namedWindow('Sem Camera',cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Sem Camera', self.front_camera)
            cv2.waitKey(1)
        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))

        while self.front_camera is None:
            time.sleep(0.01)#Just in case car takes any longer, because we need to be certain the car is done falling from the sky on spawn.

        self.episode_start = time.time()
        #self.steering_lock = False
        #self.steering_lock_start = None # this is to count time in steering lock and start penalising for long time in steering lock
        self.step_counter = 0
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        observation = self.front_camera

        return observation.astype(np.uint8), {}
    
    def step(self, action):
        self.step_counter +=1
        steer = action[0]
        throttle = action[1]
        # map steering actions
        if steer ==0:
            steer = - 0.9
        elif steer ==1:
            steer = -0.25
        elif steer ==2:
            steer = -0.1
        elif steer ==3:
            steer = -0.05
        elif steer ==4:
            steer = 0.0 
        elif steer ==5:
            steer = 0.05
        elif steer ==6:
            steer = 0.1
        elif steer ==7:
            steer = 0.25
        elif steer ==8:
            steer = 0.9
        # map throttle and apply steer and throttle
        if throttle == 0:
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=steer, brake = 1.0))
        elif throttle == 1:
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.3, steer=steer, brake = 0.0))
        elif throttle == 2:
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.7, steer=steer, brake = 0.0))
        else:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=steer, brake = 0.0))


        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))

        #distance_travelled = self.initial_location.distance(self.vehicle.get_location())

        # storing camera to return at the end in case the clean-up function destroys it
        cam = self.front_camera
        # showing image
        if self.SHOW_CAM:
            cv2.imshow('Sem Camera', cam)
            cv2.waitKey(1)

        # start defining reward from each step
        reward = 0
        done = False
        #punish for collision
        if len(self.collision_hist) != 0:
            done = True
            reward = reward - 300
            self.cleanup()
        #reward for acceleration
        if kmh < 15:
            reward = reward - 3
        elif kmh <25:
            reward = reward - 2
        elif kmh <50:
            reward = reward - 1
        elif kmh>120:
            reward = reward - 10 #punish for going too fast
        else:
            reward = reward + 1

        # check for episode duration
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            done = True
            self.cleanup()
        
        observation = cam
        
        return observation.astype(np.uint8), reward, done, False, {}

    def cleanup(self):
        for sensor in self.world.get_actors().filter('*sensor*'):      
            sensor.destroy()
        for actor in self.world.get_actors().filter('*vehicle*'):
                    
            actor.destroy()
            
        cv2.destroyAllWindows()
        
    def process_img(self, image):
        image.convert(carla.ColorConverter.CityScapesPalette)
        i = np.array(image.raw_data)
        i = i.reshape((self.im_height, self.im_width, 4))[:, :, :3] # this is to ignore the 4th Alpha channel - up to 3
        self.front_camera = i/255.0

    def collision_data(self, event):
        self.collision_hist.append(event)

In [4]:
#env = CarEnv()

#check_env(env, warn=True, skip_render_check = True) 


In [5]:
models_dir = f"models/{int(time.time())}/"
logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


env = CarEnv()

env.reset()

model = PPO('MlpPolicy', env, verbose=1,learning_rate=0.001,tensorboard_log=logdir)

TIMESTEPS = 500_000 # individual steps
iters = 0
while iters<4:  # how many training iterations you want
    iters += 1
    print('Iteration ', iters,' is to start...')
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO" )
    print('Iteration ', iters,' has been trained')
    model.save(f"{models_dir}/{TIMESTEPS*iters}")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Iteration  1  is to start...


Logging to logs/1714903411/PPO_0


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 41.6     |
|    ep_rew_mean     | -404     |
| time/              |          |
|    fps             | 18       |
|    iterations      | 1        |
|    time_elapsed    | 113      |
|    total_timesteps | 2048     |
---------------------------------


KeyboardInterrupt: 